# Tutorial on opening products from s3 bucket

In [ ]:
import json
import xarray as xr
from pathlib import Path
from xarray_eop.eop import open_eop_datatree

### Get AWS credentials

In [ ]:
# Assuming credentials are in ~/.eopf/secrets.json
try:
    SECRET_PATH = Path.home() / Path(".eopf/secrets.json")
    with open(SECRET_PATH) as f:
        secrets=json.load(f)

# Or use ENV variables
except:
    import os
    secrets = {"s3input" : {
        "key": os.getenv("AWS_ACCESS_KEY_ID"),
        "secret": os.getenv("AWS_SECRET_ACCES_KEY"),
        "endpoint_url": os.getenv("AWS_ENDPOINT_URL"),
        "region_name": os.getenv("AWS_DEFAULT_REGION")
        }
    }

secrets["s3input"].pop("region_name",None)

Setup paths

In [ ]:
product_path = "s3://s3-input/Products/NewFormat/"
auxiliary_path = "s3://s3-input/Auxiliary/"
path=auxiliary_path
path=product_path

### S3FS
Browse the s3 bucket

In [ ]:
import s3fs
s3 = s3fs.S3FileSystem(key=secrets["s3input"]["key"],
                       secret=secrets["s3input"]["secret"],
                       endpoint_url=secrets["s3input"]["endpoint_url"])
s3.ls(path)

### Open unzipped data from S3 bucket

In [ ]:
product="meas_ol2.zarr"
#Open dataset with zarr engine
# ds=xr.open_dataset(path+product,engine="zarr",backend_kwargs={"storage_options":secrets["s3input"]})
#Open dataset with custom eop engine
ds=xr.open_dataset(path+product,engine="eop",storage_options=secrets["s3input"])
ds.gifapar

### Open zipped data from S3 bucket

without the root folder, no compression
The zip archive can be obtained following:
```shell
$> cd product.zarr
$> zip -0 -r product.zarr.zip .
``` 

In [ ]:
product = path+"meas_ol2.zip"
# Open xarray dataset with zarr engine
ds = xr.open_dataset(f"zip::{product}",engine="zarr",backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
# Custom eop engine needs to be fixed
# ds=xr.open_dataset(f"zip::{product}",engine="eop",storage_options = {"s3":secrets["s3input"]})
ds

### Open a whole product from zipped S3 bucket

In [ ]:
s3_path = s3.glob(path+"S3OLCLFR*.zip")
store=f"zip::s3://{s3_path[0]}"
store

In [ ]:
dt = open_eop_datatree(store,backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
dt

### Old low-level attempts (deprecated)

In [ ]:
import zarr
import s3fs
import fsspec
s3 = s3fs.S3FileSystem(key="c0c73ed099f24d13a8f52c6b2efe0e4a",
            secret="3513bea6561c4f14bb53240937d8d4e1",
            endpoint_url="https://s3.sbg.perf.cloud.ovh.net",
            # region_name="sbg"
)
f = s3.open(product)
fs = fsspec.implementations.zip.ZipFileSystem(f,mode="r")
store = fsspec.mapping.FSMap("",fs,check=False)
store=zarr.storage.FSStore(url=f"zip://*.zarr::{product}",mode="r",s3=secrets["s3input"])
store.path
zarr.open(store,mode="r")

In [ ]:
import fsspec
fs = fsspec.filesystem("s3",key="c0c73ed099f24d13a8f52c6b2efe0e4a",
            secret="3513bea6561c4f14bb53240937d8d4e1",
            endpoint_url="https://s3.sbg.perf.cloud.ovh.net")
            # client_kwargs={"endpoint_url":"https://s3.sbg.perf.cloud.ovh.net"})
fs.glob(product)
f"zip::{product}"
print(product)
f=fsspec.open(f"zip://*.zarr::{product}",s3=secrets["s3input"])
print(f)
m=fsspec.get_mapper(f"zip://*.zarr::{product}",s3=secrets["s3input"])
print(m.fs)
zarr.open(store=m.map,mode="r")
# ds=xr.open_dataset(m,engine="zarr")